# Seq2Seq

1. Chacter level
2. Word level
3. Word levvel + Attention


**Warning**
1. 천천히 구조를 볼 것.
2. 디테일을 놓쳐도, 반드시 큰 흐름을 잡을 것


![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)


In [ ]:
!wget https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt

--2021-02-02 00:48:50--  https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9541158 (9.1M) [text/plain]
Saving to: ‘eng-fra.txt’

eng-fra.txt         100%[===================>]   9.10M  --.-KB/s    in 0.1s    

2021-02-02 00:48:50 (66.7 MB/s) - ‘eng-fra.txt’ saved [9541158/9541158]



# 전처리 - 특수문자


In [ ]:
# 단어로 분리
en_sent = "Have you had dinner?"
fr_sent = "Avez-vous déjà diné?"

en_sent = en_sent.replace('-', ' ').replace('?', ' ')
fr_sent = fr_sent.replace('-', ' ').replace('?', ' ')

print(en_sent.lower().split())
print(fr_sent.lower().split())

# ['have', 'you', 'had', 'dinner']
# ['avez', 'vous', 'déjà', 'diné']

['have', 'you', 'had', 'dinner']
['avez', 'vous', 'déjà', 'diné']


In [ ]:
with open('eng-fra.txt', 'r', encoding='utf8') as f:
    txt = f.read()

print(len(txt))

# txt 안에 들어있는 특수문자를 모두 찾아라.
print(len(set(txt)))
specials = set(txt.lower()) - set(" \n\t'abcdefghijklmnopqrstuvwxyzàâçèéêëîïòôöùúûœас")
print(sorted(specials))
print(''.join(sorted(specials)))

9395786
122
['!', '"', '$', '%', '&', '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '~', '\xa0', '«', '\xad', '»', '\u2009', '\u200b', '‐', '–', '‘', '’', '…', '\u202f', '‽', '₂', '€', '\u3000']
!"$%&()+,-./0123456789:;?~ «­» ​‐–‘’… ‽₂€　


In [ ]:
print(txt[:100])

# 특수문자 제거
txt_alpha = txt.lower()
for s in specials:
    txt_alpha = txt_alpha.replace(s, ' ')

print(txt_alpha[:100])

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Sto
go 	va  
run 	cours  
run 	courez  
wow 	ça alors  
fire 	au feu  
help 	à l'aide  
jump 	saute 
sto


In [ ]:
# eng, fra
line = [line.split('\t') for line in txt_alpha.split('\n')]
print(line[:3])

# 50000개만 추출
line = line[:50000]
print(len(line))

# 섞기
import random
random.shuffle(line)
print(line[:3])

[['go ', 'va  '], ['run ', 'cours  '], ['run ', 'courez  ']]
50000
[['she left me a note ', 'elle me laissa une note '], ["how's your boy doing ", "comment s'en sort ton garçon  "], ['you never asked why ', "vous n'avez jamais demandé pourquoi "]]


In [ ]:
a = [1, 2, 3, 4, 5]
random.shuffle(a)
print(a)

[5, 1, 2, 3, 4]


In [ ]:
# 김동균님 추천 코드 - 특수문자 제거
from string import punctuation
def preprocess_text(text):
    text = text.lower()
    symbols = punctuation
    for s in symbols:
        text = text.replace(s, ' ')
    words = text.split()
    return words

print(preprocess_text(en_sent))
print(preprocess_text(fr_sent))

['have', 'you', 'had', 'dinner']
['avez', 'vous', 'déjà', 'diné']


In [ ]:
# zip 
a = [1, 2]
b = [3, 4]
c = [5, 6]
d = [7, 8]
list(zip(a, b, c, d))
first, second = zip(a, b, c, d)
print(first)
print(second)

(1, 3, 5, 7)
(2, 4, 6, 8)


In [ ]:
# '*' oprator => unpacking
tmp = [a, b, c, d]
print(tmp)
first, second = zip(*tmp)
print(first)
print(second)

[[1, 2], [3, 4], [5, 6], [7, 8]]
(1, 3, 5, 7)
(2, 4, 6, 8)


In [ ]:
# 영어 문장, 불어 문장 분리
eng_sent, fra_sent = zip(*line)
eng_sent = [s.strip() for s in eng_sent]
fra_sent = [s.strip() for s in fra_sent]
print(eng_sent[:3])
print(fra_sent[:3])

['she left me a note', "how's your boy doing", 'you never asked why']
['elle me laissa une note', "comment s'en sort ton garçon", "vous n'avez jamais demandé pourquoi"]


# Seq2Seq I - Character Level

- (50000, 최대문장 길이, 원핫인코딩)

1. padding => 길이를 맞춰준다.
2. tokenizing => 각 문자열에 매핑되는 숫자 구성
3. indexing => token을 기준으로 실제 문장을 숫자로 변환
4. one hot encoding

In [ ]:
# 불어의 start => tab(\t), end => 줄바꿈(\n) 정의 
fra_sent = [f'\t{s}\n' for s in fra_sent]
# fra_sent = ['\t{0}\n'.format(s) for s in fra_sent]

print(eng_sent[:3])
print(fra_sent[:3])
print(len(eng_sent), len(fra_sent))

['i owe you a favor', 'what do we have to lose', 'the pain has gone']
['\tje te dois une faveur\n', "\tqu'avons nous à perdre\n", '\tla douleur est partie\n']
50000 50000


In [ ]:
# 사용되는 문자셋의 개수 - for tokenizing & indexing
eng_char = sorted(set(''.join(eng_sent)))
fra_char = sorted(set(''.join(fra_sent)))
print(f"영어 문자셋 개수: {len(eng_char)}, 불어 문자셋 개수: {len(fra_char)}")

영어 문자셋 개수: 29, 불어 문자셋 개수: 44


In [ ]:
# 가장 긴 문장의 길이를 찾아보세요. - for padding
eng_max = max([len(s) for s in eng_sent])
fra_max = max([len(s) for s in fra_sent])

eng_max = max(map(len, eng_sent))
fra_max = max(map(len, fra_sent))

print(f"영어 - 문자셋 개수: {len(eng_char)}, 최대 길이: {eng_max}")
print(f"불어 - 문자셋 개수: {len(fra_char)}, 최대 길이: {fra_max}")

영어 - 문자셋 개수: 29, 최대 길이: 24
불어 - 문자셋 개수: 44, 최대 길이: 73


## tokenizing

In [ ]:
eng_token = {c: i for i, c in enumerate(eng_char)}
fra_token = {c: i for i, c in enumerate(fra_char)}
print(eng_token)
print(fra_token)

{' ': 0, "'": 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, 'é': 28}
{'\t': 0, '\n': 1, ' ': 2, "'": 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29, 'à': 30, 'â': 31, 'ç': 32, 'è': 33, 'é': 34, 'ê': 35, 'ë': 36, 'î': 37, 'ï': 38, 'ô': 39, 'ù': 40, 'û': 41, 'œ': 42, 'с': 43}


## padding

In [ ]:
"abc".ljust(24)

'abc                     '

In [ ]:
eng_pad = [s.ljust(24) for s in eng_sent]
fra_pad = [s.ljust(73) for s in fra_sent]
print(eng_pad[:3])
print(fra_pad[:3])


['i owe you a favor       ', 'what do we have to lose ', 'the pain has gone       ']
['\tje te dois une faveur\n                                                  ', "\tqu'avons nous à perdre\n                                                 ", '\tla douleur est partie\n                                                  ']


## indexing

In [ ]:
eng_idx = [list(map(eng_token.get, s)) for s in eng_pad]
fra_idx = [list(map(fra_token.get, s)) for s in fra_pad]

print(eng_idx[:3])
print(fra_idx[:3])

[[10, 0, 16, 24, 6, 0, 26, 16, 22, 0, 2, 0, 7, 2, 23, 16, 19, 0, 0, 0, 0, 0, 0, 0], [24, 9, 2, 21, 0, 5, 16, 0, 24, 6, 0, 9, 2, 23, 6, 0, 21, 16, 0, 13, 16, 20, 6, 0], [21, 9, 6, 0, 17, 2, 10, 15, 0, 9, 2, 20, 0, 8, 16, 15, 6, 0, 0, 0, 0, 0, 0, 0]]
[[0, 13, 8, 2, 23, 8, 2, 7, 18, 12, 22, 2, 24, 17, 8, 2, 9, 4, 25, 8, 24, 21, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [0, 20, 24, 3, 4, 25, 18, 17, 22, 2, 17, 18, 24, 22, 2, 30, 2, 19, 8, 21, 7, 21, 8, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [0, 15, 4, 2, 7, 18, 24, 15, 8, 24, 21, 2, 8, 22, 23, 2, 19, 4, 21, 23, 12, 8, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]


## one hot encoding

In [ ]:
import tensorflow as tf

eng_onehot = tf.keras.utils.to_categorical(eng_idx, num_classes=29)
fra_onehot = tf.keras.utils.to_categorical(fra_idx, num_classes=44)

print(eng_onehot.shape, fra_onehot.shape)

(50000, 24, 29) (50000, 73, 44)


## model seq2seq

In [ ]:
# 영어 - 문자셋 개수: 29, 최대 길이: 24, (50000, 24, 29)
# 불어 - 문자셋 개수: 44, 최대 길이: 73, (50000, 73, 44)

import tensorflow as tf

enc_X = tf.keras.layers.Input(shape=[24, 29])
enc_H, enc_S = tf.keras.layers.GRU(256, return_state=True)(enc_X)

dec_X = tf.keras.layers.Input(shape=[72, 44])
dec_H, dec_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(dec_X, initial_state=enc_S)

dec_H = tf.keras.layers.Dense(256, activation='swish')(dec_H)
dec_Y = tf.keras.layers.Dense(44, activation='softmax')(dec_H)

model = tf.keras.Model([enc_X, dec_X], dec_Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 24, 29)]     0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 72, 44)]     0                                            
__________________________________________________________________________________________________
gru_5 (GRU)                     [(None, 256), (None, 220416      input_7[0][0]                    
__________________________________________________________________________________________________
gru_6 (GRU)                     [(None, 72, 256), (N 231936      input_8[0][0]                    
                                                                 gru_5[0][1]                

In [ ]:
model.fit([eng_onehot, fra_onehot[:, :-1]], fra_onehot[:, 1:],
          epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
313/313 [==============================] - 5s 17ms/step - loss: 0.2984 - accuracy: 0.9064 - val_loss: 0.2952 - val_accuracy: 0.9068
Epoch 2/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2879 - accuracy: 0.9095 - val_loss: 0.2898 - val_accuracy: 0.9090
Epoch 3/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2793 - accuracy: 0.9121 - val_loss: 0.2873 - val_accuracy: 0.9095
Epoch 4/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2713 - accuracy: 0.9145 - val_loss: 0.2819 - val_accuracy: 0.9115
Epoch 5/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2641 - accuracy: 0.9166 - val_loss: 0.2782 - val_accuracy: 0.9128
Epoch 6/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2574 - accuracy: 0.9185 - val_loss: 0.2757 - val_accuracy: 0.9134
Epoch 7/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2514 - accuracy: 0.9203 - val_loss: 0.2721 - val_accuracy: 0.9149

In [ ]:
# 영어 - 문자셋 개수: 29, 최대 길이: 24, (50000, 24, 29)
# 불어 - 문자셋 개수: 44, 최대 길이: 73, (50000, 73, 44)
import numpy as np

def translate(eng):
    # eng => padding => indexing => onehot
    eng_sent = list(eng.ljust(24))
    eng_idx = np.array([[eng_token.get(c) for c in eng_sent]])
    eng_onehot = tf.keras.utils.to_categorical(eng_idx, num_classes=29)
    eng_onehot = eng_onehot.reshape(1, 24, 29)
    # shape => (1. 24, 29)

    fra = ''
    for n in range(72):
        # fra => padding => indexing => onehot
        fra_sent = list(f'\t{fra}'.ljust(72))
        fra_idx = np.array([[fra_token.get(c) for c in fra_sent]])
        fra_onehot = tf.keras.utils.to_categorical(fra_idx, num_classes=44)
        fra_onehot = fra_onehot.reshape(1, 72, 44)
        # shape => (1. 72, 44)

        fra_next = model.predict([eng_onehot, fra_onehot])
        # result (1, 72, 44)

        fra = [fra_char[i] for i in np.argmax(fra_next[0], axis=1)]
        # print(''.join(fra))
        fra = fra[:n+1]
        # print(''.join(fra))
        fra = ''.join(fra)

        if fra[-1] == '\n':
            break

    return fra


In [ ]:
import random

# 셔플
indices = list(range(50000))
random.shuffle(indices)

# 10개 추출
for n in indices[:10]:
    print(f'영어: {eng_sent[n]}\n불어: {fra_sent[n][1:-1]}')
    print(f'번역: {translate(eng_sent[n])}')

영어: was it an explosion
불어: était ce une explosion
번역: étais je en train de te prendre

영어: i'm being held hostage
불어: je suis retenue en otage
번역: je suis contente de vous aider

영어: he used the dictionary
불어: il utilisa le dictionnaire
번역: il a été toujours en train de me render chez moi

영어: don't read my journal
불어: ne lisez pas ma revue
번역: ne me fais pas de partir de moi

영어: those are my cds
불어: ce sont mes cds
번역: ce sont les maths

영어: tom is not a criminal
불어: tom n'est pas un criminel
번역: tom n'est pas un peu de maintenant

영어: tom lost his umbrella
불어: tom a perdu son parapluie
번역: tom a perdu les chiens autres secret

영어: sit wherever you like
불어: asseyez vous où ça vous chante
번역: fais moi juste ce que je veux dire

영어: we're tempting fate
불어: nous tentons le diable
번역: nous sommes toutes deux messages

영어: you have a lot of books
불어: vous avez beaucoup de livres
번역: tu as l'air déprimées



# Seq2Seq II - Word Levelp

## 단어를 기준으로 문장을 분리한다.

In [ ]:
print(eng_sent[:3])
print(fra_sent[:3])

eng_word = [s.split() for s in eng_sent]
fra_word = [f'<sos> {s} <eos>'.split() for s in fra_sent]

print(eng_word[:3])
print(fra_word[:3])

['she left me a note', "how's your boy doing", 'you never asked why']
['elle me laissa une note', "comment s'en sort ton garçon", "vous n'avez jamais demandé pourquoi"]
[['she', 'left', 'me', 'a', 'note'], ["how's", 'your', 'boy', 'doing'], ['you', 'never', 'asked', 'why']]
[['<sos>', 'elle', 'me', 'laissa', 'une', 'note', '<eos>'], ['<sos>', 'comment', "s'en", 'sort', 'ton', 'garçon', '<eos>'], ['<sos>', 'vous', "n'avez", 'jamais', 'demandé', 'pourquoi', '<eos>']]


## Tokenizing, Indexing, Padding, Onehot(?)

In [ ]:
import tensorflow as tf

# tokenizing
tokenizer_en = tf.keras.preprocessing.text.Tokenizer()
tokenizer_en.fit_on_texts(eng_word)
tokenizer_fr = tf.keras.preprocessing.text.Tokenizer()
tokenizer_fr.fit_on_texts(fra_word)

# indexing
eng_idx = tokenizer_en.texts_to_sequences(eng_word)
fra_idx = tokenizer_fr.texts_to_sequences(fra_word)
print(eng_idx[:3])
print(fra_idx[:3])

# padding
eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_idx, padding='post')
fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_idx, padding='post')
print(eng_pad[:2])
print(fra_pad[:2])
print(eng_pad.shape, fra_pad.shape)


[[29, 171, 10, 3, 1272], [390, 22, 273, 205], [2, 101, 329, 58]]
[[1, 29, 24, 1632, 25, 746, 2], [1, 58, 303, 1740, 67, 277, 2], [1, 4, 399, 98, 336, 64, 2]]
[[  29  171   10    3 1272    0    0    0]
 [ 390   22  273  205    0    0    0    0]]
[[   1   29   24 1632   25  746    2    0    0    0    0    0    0    0
     0    0]
 [   1   58  303 1740   67  277    2    0    0    0    0    0    0    0
     0    0]]
(50000, 8) (50000, 16)


In [ ]:
print(tokenizer_en.word_index)
print(tokenizer_fr.word_index)

print("영어 단어 개수: {0}".format(len(tokenizer_en.word_index) + 1))
print("불어 단어 개수: {0}".format(len(tokenizer_fr.word_index) + 1))

{'i': 1, 'you': 2, 'a': 3, 'is': 4, 'to': 5, 'the': 6, 'it': 7, 'he': 8, "i'm": 9, 'me': 10, 'do': 11, 'tom': 12, 'are': 13, "don't": 14, 'this': 15, 'not': 16, "you're": 17, 'have': 18, 'that': 19, 'we': 20, 'my': 21, 'your': 22, 'was': 23, 'did': 24, 'all': 25, 'be': 26, 'of': 27, 'what': 28, 'she': 29, "it's": 30, 'like': 31, 'go': 32, 'they': 33, 'can': 34, 'in': 35, 'want': 36, 'very': 37, 'how': 38, 'no': 39, 'him': 40, 'here': 41, "can't": 42, 'for': 43, "we're": 44, 'on': 45, 'know': 46, 'get': 47, "i'll": 48, 'up': 49, "that's": 50, 'need': 51, 'out': 52, 'his': 53, 'at': 54, 'just': 55, 'one': 56, 'so': 57, 'why': 58, 'there': 59, 'her': 60, 'with': 61, "he's": 62, 'now': 63, 'think': 64, 'good': 65, 'please': 66, 'will': 67, 'come': 68, 'see': 69, 'too': 70, 'has': 71, 'got': 72, 'am': 73, 'love': 74, 'were': 75, 'help': 76, 'had': 77, 'really': 78, 'look': 79, "didn't": 80, 'us': 81, 'right': 82, 'who': 83, 'about': 84, "they're": 85, 'feel': 86, 'take': 87, 'let': 88, 'an'

In [ ]:
# 영어 단어 개수: 6090, (50000, 8) 
# 불어 단어 개수: 11804, (50000, 16)

In [ ]:
# onehot ? => Embedding
# eng_onehot = tf.keras.utils.to_categorical(eng_pad, num_classes=6090)

## 모델 with Embedding Layer

In [ ]:
6090 * 40

243600

In [ ]:
# 영어 단어 개수: 6090, (50000, 8) 
# 불어 단어 개수: 11804, (50000, 16)

import tensorflow as tf

enc_X = tf.keras.layers.Input(shape=[8])
enc_E = tf.keras.layers.Embedding(6090, 40)(enc_X)
enc_H, enc_S = tf.keras.layers.GRU(256, return_state=True)(enc_E)

dec_X = tf.keras.layers.Input(shape=[15])
dec_E = tf.keras.layers.Embedding(11804, 40)(dec_X)
dec_H, dec_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(dec_E, initial_state=enc_S)

dec_H = tf.keras.layers.Dense(256, activation='swish')(dec_H)
dec_Y = tf.keras.layers.Dense(11804, activation='softmax')(dec_H)

model = tf.keras.Model([enc_X, dec_X], dec_Y)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 8, 40)        243600      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 15, 40)       472160      input_6[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.fit([eng_pad, fra_pad[:, :-1]], fra_pad[:, 1:], 
          epochs=50, batch_size=128, validation_split=0.2)

In [ ]:
# 영어 단어 개수: 6090, (50000, 8) 
# 불어 단어 개수: 11804, (50000, 16)
import numpy as np

def translate(eng):
    # eng => index => pad
    eng_idx = tokenizer_en.texts_to_sequences([eng])
    eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_idx, maxlen=8, padding='post')

    fra = []
    for n in range(15):
        # fra => index => pad
        fra_idx = tokenizer_fr.texts_to_sequences([['<sos>'] + fra])
        fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_idx, maxlen=15, padding='post')
        
        fra_next = model.predict([eng_pad, fra_pad])

        fra = [tokenizer_fr.index_word[i] for i in np.argmax(fra_next[0], axis=1) if i > 0]
        fra = fra[:n+1]

        if fra[-1] == '<eos>':
            break

    return fra

In [ ]:
import random

# 셔플
indices = list(range(50000))
random.shuffle(indices)

# 10개 추출
for n in indices[:10]:
    print(f"영어: {' '.join(eng_word[n])}\n불어: {' '.join(fra_word[n][1:-1])}")
    print(f"번역: {' '.join(translate(eng_word[n])[:-1])}")
    print()

영어: that's very funny
불어: c'est très marrant
번역: c'est très drôle

영어: i'm all out of ideas
불어: je suis dépourvu d'idées
번역: je suis dépourvu d'idées

영어: a lot of fish perished
불어: de nombreux poissons périrent
번역: de nombreux poissons ça était prenez en êtes

영어: is this seat taken
불어: cette place est elle libre
번역: cette place est elle libre

영어: call home
불어: appelle à la maison
번역: appelle à la maison

영어: where's the red cross
불어: où est la croix rouge
번역: où se trouve le pont

영어: you're very forward
불어: vous êtes très effrontée
번역: vous êtes fort effronté

영어: it's merely a joke
불어: ce n'est qu'une plaisanterie
번역: ce n'est qu'une mort

영어: where is your house
불어: où est votre maison
번역: où est votre maison

영어: stop talking loudly
불어: arrête de parler fort
번역: arrêtez de parler fort



### 잠깐 RNN 리턴에 대한 이해를 위한 보충

In [ ]:

# (5000, 7, 5)
enc_H = tf.keras.layers.GRU(256)(enc_E)
# enc_H => (5000, 256)

# (5000, 7, 5)
enc_H = tf.keras.layers.GRU(256. return_sequences=True)(enc_E)
# enc_H => (5000, 7, 256)

# (5000, 7, 5)
enc_H, enc_S = tf.keras.layers.GRU(256. return_state=True)(enc_E)
# enc_H => (5000, 256), enc_S => (5000, 256)

# (5000, 7, 5)
enc_H, enc_S = tf.keras.layers.GRU(256. return_sequences=True. return_state=True)(enc_E)
# enc_H => (5000, 7, 256), enc_S => (5000, 256)

## 모델링: seq2seq model - Attention 구조 (dot prodoct ver)
- reference: https://wikidocs.net/22893

In [ ]:
# 영어 단어 개수: 6090, (50000, 8) 
# 불어 단어 개수: 11804, (50000, 16)

import tensorflow as tf

enc_X = tf.keras.layers.Input(shape=[8])
enc_E = tf.keras.layers.Embedding(6090, 40)(enc_X)
enc_H, enc_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(enc_E)
# shape => (50000, 8, 256)

dec_X = tf.keras.layers.Input(shape=[15])
dec_E = tf.keras.layers.Embedding(11804, 40)(dec_X)
dec_H, dec_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(dec_E, initial_state=enc_S)
# shape => (50000, 15, 256)

# key, value, query
key = tf.keras.layers.Dense(64)(enc_H) # shape => (50000, 8, 64)
value = tf.keras.layers.Dense(64)(enc_H) # shape => (50000, 8, 64)
query = tf.keras.layers.Dense(64)(dec_H) # shape => (50000, 15, 64)

# attention
score = tf.matmul(query, key, transpose_b=True) # (50000, 15, 64) * (50000, 64, 8) => (50000, 15, 8)
score = tf.nn.softmax(score, axis=-1) # (50000, 15, 8)
attention = tf.matmul(score, value) # (50000, 15, 8) * (50000, 8, 64) => (50000, 15, 64)

dec_H = tf.keras.layers.Concatenate()([attention, dec_H])

dec_H = tf.keras.layers.Dense(256, activation='swish')(dec_H)
dec_Y = tf.keras.layers.Dense(11804, activation='softmax')(dec_H)

model = tf.keras.Model([enc_X, dec_X], dec_Y)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 8, 40)        243600      input_14[0][0]                   
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 15, 40)       472160      input_15[0][0]                   
____________________________________________________________________________________________

In [ ]:
model.fit([eng_pad, fra_pad[:, :-1]], fra_pad[:, 1:], 
          epochs=50, batch_size=128, validation_split=0.2)

Epoch 1/50
313/313 [==============================] - 13s 29ms/step - loss: 2.8162 - accuracy: 0.6476 - val_loss: 1.8852 - val_accuracy: 0.7209
Epoch 2/50
313/313 [==============================] - 8s 27ms/step - loss: 1.7616 - accuracy: 0.7302 - val_loss: 1.6100 - val_accuracy: 0.7504
Epoch 3/50
313/313 [==============================] - 8s 27ms/step - loss: 1.5323 - accuracy: 0.7549 - val_loss: 1.4836 - val_accuracy: 0.7669
Epoch 4/50
313/313 [==============================] - 8s 27ms/step - loss: 1.4094 - accuracy: 0.7698 - val_loss: 1.4345 - val_accuracy: 0.7749
Epoch 5/50
313/313 [==============================] - 8s 27ms/step - loss: 1.3222 - accuracy: 0.7800 - val_loss: 1.3681 - val_accuracy: 0.7819
Epoch 6/50
313/313 [==============================] - 8s 27ms/step - loss: 1.2614 - accuracy: 0.7874 - val_loss: 1.3117 - val_accuracy: 0.7883
Epoch 7/50
313/313 [==============================] - 8s 27ms/step - loss: 1.2069 - accuracy: 0.7933 - val_loss: 1.2744 - val_accuracy: 0.792